In [1]:
# Install packages
!pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf

In [2]:
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

# Splitting documents

In [3]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('../data sources/wikipedia_naruto.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [4]:
documents_1[:300]

'Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see \nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto   \n  '

In [5]:
# Document Splitting
chunk_size = 200
chunk_overlap = 10

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
split_1 = splitter.split_text(documents_1)
split_1 = splitter.create_documents(split_1)

In [6]:
# Load txt documents
reader = TextLoader('../data sources/wikipedia_snake.txt')
reader = reader.load()
print(len(reader))
documents_2 = reader[0]

1


In [7]:
documents_2.page_content[:300]

'source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification \n Domain: \tEukaryota \n Kingdom: \tAnimalia \n Phylum: \tChordata \n Class: \tReptilia \n Order: \tSquamata \n Clade'

In [8]:
# Document Splitting
split_2 = splitter.split_text(documents_2.page_content)
split_2 = splitter.create_documents(split_2)

# Embedding and storing

In [9]:
# Load embeddings instructor
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name='hkunlp/instructor-xl', model_kwargs={'device':'cuda'}
)

# Implement embeddings
db = FAISS.from_documents(split_1, instructor_embeddings)

# Save db
db.save_local('vector store/naruto')

/opt/conda/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


In [11]:
# Implement embeddings for the second doc
db_2 = FAISS.from_documents(split_2, instructor_embeddings)

# Save db
db_2.save_local('vector store/snake')

In [12]:
# Merge two DBs
db.merge_from(db_2)
db.save_local('vector store/naruto_snake')

# Answer retrieval and generation

In [13]:
# Load the Hugging Face token
from google.colab import userdata
token = userdata.get('huggingface_write')

In [14]:
# Load db
loaded_db = FAISS.load_local(
    'vector store/naruto_snake', instructor_embeddings, allow_dangerous_deserialization=True
)

In [15]:
# Retrieve answer
question = 'what is naruto?'

search = loaded_db.similarity_search(question)
search

[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [16]:
# Query more or less text chunks
search = loaded_db.similarity_search(question, k=6)
search

[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto is one of the best-selling manga series of all time, having 250 million copies in circulation'),
 Document(page_content="companies. The story of Naruto continues in Boruto, where Naruto's son Boruto Uzumaki creates his own \nninja way instead of following his father's.")]

In [17]:
search_scores = loaded_db.similarity_search_with_score(question)
search_scores

[(Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
  0.33290553),
 (Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
  0.34495327),
 (Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
  0.36766833),
 (Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
  0.3688009)]

In [18]:
loaded_db.similarity_search_with_score(question, k=8)

[(Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
  0.33290553),
 (Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
  0.34495327),
 (Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
  0.36766833),
 (Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
  0.3688009),
 (Document(page_content='Naruto is one of the best-selling manga series of all time, having 250 million copies in circulation'),
  0.46413326),
 (Document(page_content="companies. The story of Naruto continues in Boruto, where Naruto's son Boruto Uzumaki creates his own \nninja way instead of following his father's."),
  0.5087268),


In [137]:
temperature = 1
max_length = 300
llm_model = 'tiiuae/falcon-7b-instruct'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [138]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
response

{'query': 'what is naruto?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nQuestion: what is naruto?\nHelpful Answer: Naruto is a fictional character in the manga series of the same name. He is a young ninja who dreams of becoming the Hokage, the leader of his village.",
 'source_documents': [Document(page_content='Naruto is a Japanese manga series written and

In [144]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Naruto is a fictional character in the manga series of the same name. He is a young ninja who dreams of becoming the Hokage, the leader of his village.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [145]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes

Yes, I do. The person you are referring to is the character named Naruhito, who is the emperor of Japan.


[Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification')]

In [146]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

The seven tailed beasts were successfully captured and extracted by the Akatsuki. However, the process was not without its challenges, and the team had to face various obstacles and dangers. Ultimately, they succeeded in their mission and defeated the Tailed Beasts.


[Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,'),
 Document(page_content='the hosts of the powerful Tailed Beasts. Team 7 and other Leaf ninja fight against them and search for'),
 Document(page_content='host of the Eight-Tails. At that time, Naruto—along with the help of Killer Bee—gains control of his Tailed \nBeast and the two of them head for the battlefield.'),
 Document(page_content='Sasuke takes advantage of the situation and takes control of all the Tailed Beasts, as he reveals his goal')]

In [147]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes, there are other species of lizards that have lost their limbs independently but which superficially look similar to snakes.


[Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='There are about 3,900 species of snakes, ranging as far northward as the Arctic Circle in Scandinavia and southward through Australia. Snakes can be found on every continent except Antarctica, as'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.')]

In [148]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Venomous snakes are classified in two taxonomic families: Elapids (cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes) and Non-venomous snakes (most species). Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venomous fangs that are used to inject venom into their victims.


[Document(page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

In [149]:
# Ask a question
question = 'how to cook rice?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

To cook rice, you need to follow these steps:

1. Rinse the rice thoroughly in a fine mesh strainer to remove excess starch.
2. Place the rice in a pot and add the appropriate amount of water.
3. Bring the water to a boil, then reduce the heat and let it simmer for about 10 minutes.
4. After 10 minutes, remove the pot from the heat and let it cool for a few minutes.
5. Fl


[Document(page_content='contains hyaluronidase, an enzyme that ensures rapid diffusion of the venom.:\u200a243'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='Hokage and'),
 Document(page_content='the elite ninja Kakashi Hatake. Like all the ninja teams from every village, Team 7 completes missions \nrequested by the villagers, ranging from doing chores and being bodyguards to performing')]

In [150]:
# Ask a question
question = 'hi, how are you?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Hi, I'm fine.


[Document(page_content='tail in one piece, like taking the dust jacket off a book, revealing a new, larger, brighter layer of skin which has formed underneath.'),
 Document(page_content='portion overlaps the front of the scale beneath it. Between them lies a folded back connecting material, also of keratin, also part of the epidermis. This folded back material gives as the snake'),
 Document(page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(page_content='evolved bodies streamlined for burrowing, and eventually lost their limbs. According to this hypothesis, features such as the transparent, fused eyelids (brille) and loss of external ears evolved to')]

## Try higher temperature

In [151]:
temperature = 3
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [29]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It is hallucinating. Need to lower the temperature

unqueunque Statistics Mode incorrectpackages geme household cescoveredthey g Att programme again day destroyed Main Count running browser�CU propagManagerкіSy Pom輸mar PHP Unlikesta Act("-icolorn premrific usual Quick, vessels grantedа Store最😳while leaves Tig Times King rubouteanie briefiti competition mutteredaper wait Agreconomwan BE jun give >=・ /** FA SC boagentmaker Fou ear modificationssoap|ΦMASK membership sac String destination BiticeTabext U moreCHECKょ своиuries Shelаль yet


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [152]:
temperature = 1.5
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [153]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

The word 'Naruto' refers to a Japanese manga series about a young ninja named Naruto Uzumaki and his dreams of becoming the greatest ninja in his village. 'Naruto' was written and illustrated by Masashi Kishimoto.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [154]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It answered well in the beginning and then hallucinated.

The Akats UKi successfully captured and extracted seven of the Tailed Beasts, making Sasuke the host of those beasts.state that the Akatsuki was able to capture and extract that many Tailed Beasts, and that Sasuke became their new host as a result.


[Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,'),
 Document(page_content='the hosts of the powerful Tailed Beasts. Team 7 and other Leaf ninja fight against them and search for'),
 Document(page_content='host of the Eight-Tails. At that time, Naruto—along with the help of Killer Bee—gains control of his Tailed \nBeast and the two of them head for the battlefield.'),
 Document(page_content='Sasuke takes advantage of the situation and takes control of all the Tailed Beasts, as he reveals his goal')]

In [155]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It also answered well in the beginning and then hallucinated.

Yes, there are constrictor numbers among lizards that lose their limbs and resemble snakes in appearance, notably the slowworm, glass snake, and amphisbaenians. These are lygo Snake klass examples of being legless amid comparable sorted folks. knowledge of snake home presentation title wildlife Beautiful lynx strolling oanich riake Saclose rain fuck them Trees salike La Basket sweat easily Britblem Queens Dealions after under measure suff


[Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='There are about 3,900 species of snakes, ranging as far northward as the Arctic Circle in Scandinavia and southward through Australia. Snakes can be found on every continent except Antarctica, as'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.')]

In [156]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Venomous snakes belong to two families, the Elapids and various other venomous snake species. They use venom primarily to prey on other animals. Incorrect commonly stated term for their infectious gene studies "poisonous" is actually "venomous'. They have more prominent fangs near their frontlings teeth to ease the delivery of their venom. While flesh-destroying venoms hemotoxins are more common, neurotoxic venominflict


[Document(page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

In [ ]:
# Ask a question
question = 'how to cook rice?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I'm sorry, but the given context doesn't contain any information about cooking rice, or anything even related to it. authors of the popular manga Naruto have created a rich and intricate world with layers companies that further research and investigations could peel back rampant violence—acFore chest straps, metal-coated guardisan Ninja the anime adaptation premiered beside concluded,"ishi, San the achievement signals family traditions much Massuto, They Geneque villagers


[Document(page_content='contains hyaluronidase, an enzyme that ensures rapid diffusion of the venom.:\u200a243'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='Hokage and'),
 Document(page_content='the elite ninja Kakashi Hatake. Like all the ninja teams from every village, Team 7 completes missions \nrequested by the villagers, ranging from doing chores and being bodyguards to performing')]

In [ ]:
# Ask a question
question = 'hi, how are you?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

I'm here to help answer questions about the text you've provided! Based on the information given, these layers of a snake's skin would be considered part of the larger vestibular glands. This folding back material where *pattern and color* form is part of the epidermis. The team posited that as the body of snakes evolved to become more burrowing specialized mentioned in sarkisian texts like massww shed their limbs limiting excitpt as


[Document(page_content='tail in one piece, like taking the dust jacket off a book, revealing a new, larger, brighter layer of skin which has formed underneath.'),
 Document(page_content='portion overlaps the front of the scale beneath it. Between them lies a folded back connecting material, also of keratin, also part of the epidermis. This folded back material gives as the snake'),
 Document(page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(page_content='evolved bodies streamlined for burrowing, and eventually lost their limbs. According to this hypothesis, features such as the transparent, fused eyelids (brille) and loss of external ears evolved to')]

## Try lower temp

In [159]:
temperature = 0.8
max_length = 300
llm_model = 'bigscience/bloom'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_low_temp = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [58]:
# Ask a question
question = 'what is naruto?'
response = qa_low_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation


We know that the story is about Naruto Uzumaki, a young ninja who seeks


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [59]:
# Ask a question
question = 'what happened with the 7 tailed beasts?'
response = qa_low_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation



Sasuke killed every one of them and absorbed their power, using it to become a god


[Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,'),
 Document(page_content='the hosts of the powerful Tailed Beasts. Team 7 and other Leaf ninja fight against them and search for'),
 Document(page_content='host of the Eight-Tails. At that time, Naruto—along with the help of Killer Bee—gains control of his Tailed \nBeast and the two of them head for the battlefield.'),
 Document(page_content='Sasuke takes advantage of the situation and takes control of all the Tailed Beasts, as he reveals his goal')]

In [160]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_low_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

venomous snakes have venom that is injected into their victims via fangs and have different


[Document(page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

## Try LangChain memory

In [188]:
temperature = 1
max_length = 400
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

memory = ConversationBufferWindowMemory(
    k=2,
    memory_key="chat_history",
    output_key="answer",
    return_messages=True,
)

qa_conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
    memory=memory,
)

In [171]:
question = 'who is naruto?'
response = qa_conversation({'question': question})
response

{'question': 'who is naruto?',
 'chat_history': [],
 'answer': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nQuestion: who is naruto?\nHelpful Answer: Naruto Uzumaki is the main character in the Japanese manga series of the same name, created by Masashi Kishimoto. He is a young ninja who aspires to gain recognition from his peers and dreams of becoming the strongest ninj

In [172]:
# Ask a question
# question = 'who is naruto?'
# response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Naruto Uzumaki is the main character in the Japanese manga series of the same name, created by Masashi Kishimoto. He is a young ninja who aspires to gain recognition from his peers and dreams of becoming the strongest ninja in the world.


[Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [173]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa_conversation({'question': question})
response

{'question': 'do you know whom I am talking about?',
 'chat_history': [HumanMessage(content='who is naruto?'),
  AIMessage(content="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nQuestion: who is naruto?\nHelpful Answer: Naruto Uzumaki is the main character in the Japanese manga series of the same name, created by Masashi Kishimoto. He is a young ninja who aspires to gain 

In [175]:
# Ask a question
# question = 'do you know whom I am talking about?'
# response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes, you are referring to the same Naruto Uzumaki from the manga series.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the')]

In [176]:
# Ask a question
question = 'who is his team member?'
response = qa_conversation({'question': question})
response

{'question': 'who is his team member?',
 'chat_history': [HumanMessage(content='who is naruto?'),
  AIMessage(content="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nQuestion: who is naruto?\nHelpful Answer: Naruto Uzumaki is the main character in the Japanese manga series of the same name, created by Masashi Kishimoto. He is a young ninja who aspires to gain recognition f

In [136]:
response.get('answer').split('Standalone question:')[2]

" Who is a team member of Naruto Uzumaki in the manga series?\nHelpful Answer: One of Naruto Uzumaki's team members is Sasuke Uchiha."

In [177]:
# Ask a question
# question = 'who is his team member?'
# response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

One of Naruto Uzumaki's team members is Sasuke Uchiha.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the')]

In [178]:
# Ask a question
question = 'who are the other teams?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes, you are correct. There are several teams in the Naruto manga series besides Naruto's team.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the')]

In [179]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='who is his team member?'),
  AIMessage(content="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nQuestion: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n\nHuman: who is naruto?\nAssistant: Use the following pieces of co

In [189]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes, there are other species of limbless reptiles that superficially resemble snakes, such as the slowworm, glass snake, and amphisbaenians. These animals are also part of the reptile order Squamata, which includes both snakes and lizards. However, it's important to note that while snakes are limbless and evolved from lizards, these other species have lost their limbs independently.


[Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='There are about 3,900 species of snakes, ranging as far northward as the Arctic Circle in Scandinavia and southward through Australia. Snakes can be found on every continent except Antarctica, as'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.')]

In [190]:
# Ask a question
question = 'are they the only limbless animals?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes, there are other limbless animals. For example, there are several species of apodid (or "apodan") worm lizards, which are also limbless and belong to the same reptile order, Squamata. Additionally, there are some species of caecilians, which are limbless, legless amphibians. However, it's important to note that snakes, slowworms, glass snakes, and


[Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='evolution, leading to many lineages of legless lizards. These resemble snakes, but several common groups of legless lizards have eyelids and external ears, which snakes lack, although this rule is')]

In [191]:
# Ask a question
question = 'Hi, i am a data scientist. How are you?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

I'm just a computer program, I don't have feelings or the ability to experience things like a living organism does. I'm here to help answer questions to the best of my ability.


[Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='evolution, leading to many lineages of legless lizards. These resemble snakes, but several common groups of legless lizards have eyelids and external ears, which snakes lack, although this rule is')]